In [1]:
from source.class_yolo_trainer import YOLOTrainer

dataset_constructor = YOLOTrainer
dataset_constructor = YOLOTrainer()

In [ ]:
dataset_constructor.image_folder = "pelotas_19-08/images"
dataset_constructor.annotations_folder = "pelotas_19-08/labels"
dataset_constructor.yolo_Classes = "Pellet"
dataset_constructor.test_percentual_divisor = 20
dataset_constructor.slicing()

In [ ]:
dataset_constructor.training()

In [2]:
dataset_constructor.train_model = "train16"
dataset_constructor.object_to_predict = "D:/Judson_projetos/Yolo_trainer/2024-07-08_12.15.50.png"
dataset_constructor.predict_confidence = 0.1
dataset_constructor.predict()


image 1/1 D:\Judson_projetos\Yolo_trainer\2024-07-08_12.15.50.png: 544x960 3 Pelotas, 45.6ms
Speed: 4.0ms preprocess, 45.6ms inference, 78.6ms postprocess per image at shape (1, 3, 544, 960)
D:/Judson_projetos/Yolo_trainer/2024-07-08_12.15.50.png


In [ ]:
from utils.check_file import check_file

check_file("any_file")

In [ ]:
from utils.torch_is_available import torch_is_available

torch_is_available()

In [ ]:
from utils.cvat_dataset import cvat_dataset

cvat_dataset("pelotas_19-08", "pelotas_19-08")

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov8n.pt")

# # Run batched inference on a list of images
# results = model("vid_1.mp4")  # return a generator of Results objects
# # Process results generator
# i = 0
# for result in results:
#     device = result.cuda()
#     boxes = result.boxes  # Boxes object for bounding box outputs
#     conf = boxes.conf
#     cls = boxes.cls
#     fmt = boxes.xywhn
#     masks = result.masks  # Masks object for segmentation masks outputs
#     keypoints = result.keypoints  # Keypoints object for pose outputs
#     probs = result.probs  # Probs object for classification outputs
#     obb = result.obb  # Oriented boxes object for OBB outputs
#     # result.show()  # display to screen
#     result.save(filename=f"result{i}.jpg")  # save to disk
#     i += 1